In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [42]:
# data set loadinng
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
# Preprocessing
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
# Encoding the categorical variable
lebal_encoded_gender=LabelEncoder()
df['Gender']=lebal_encoded_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [45]:
# onehot encoding gerography veriable
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geo=OneHotEncoder()
geo_encoder=onehot_encode_geo.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [46]:
onehot_encode_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [47]:
# applying toarray will sparce the value after onehotencoding
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [48]:
# combining Data
df=pd.concat([df.drop('Geography',axis=1), geo_encoded_df],axis=1)
df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [49]:
df.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [ ]:
# saving the encoders into pickle files 
with open('label_encoder_gender.pkl','wb') as file:  # wb : write bite mode
    pickle.dump(lebal_encoded_gender,file)

with open('onehot_encoder_geo.plk','wb') as file:
    pickle.dump(onehot_encode_geo,file)



In [51]:
# Diveiding Data into independent and dependent features
X=df.drop('Exited', axis=1) #Independent variables
y=df['Exited'] #dependent variable

# spliting Data into Train and test
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2,random_state=42)

# scalling the Features 
scaller=StandardScaler()
X_train=scaller.fit_transform(X_train)
X_test=scaller.transform(X_test)

In [52]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [53]:
# saving scaler into pickle file
with open('scaler.pkl','wb') as file:
    pickle.dump(scaller,file)

# ANN implementation

In [54]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime

In [55]:
# Building ANN model
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL 1 which connected to input layer
    Dense(32,activation='relu'), # HL 2
    Dense(1, activation='sigmoid') # output layer

]
)

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01) # optimizer : using adam optimizer
loss=tensorflow.keras.losses.BinaryCrossentropy() #Loss function
loss

In [58]:
# complie model
model.compile(optimizer=opt,loss="binary_crossentropy", metrics=['accuracy'])

## Creating Tensorboard

In [59]:
# Setups the Tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [60]:
# setup the early stopinng
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

In [61]:
# Training model

history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4061 - accuracy: 0.8286 - val_loss: 0.3540 - val_accuracy: 0.8520
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3545 - accuracy: 0.8539 - val_loss: 0.3456 - val_accuracy: 0.8560
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3466 - accuracy: 0.8570 - val_loss: 0.3386 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3439 - accuracy: 0.8593 - val_loss: 0.3361 - val_accuracy: 0.8645
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3394 - accuracy: 0.8614 - val_loss: 0.3405 - val_accuracy: 0.8620
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3364 - accuracy: 0.8634 - val_loss: 0.3516 - val_accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.8619 - val_loss: 0.3427 - val_accuracy: 0.8585

In [62]:
model.save("model.h5")

e:\Gen AI Course\DL for NLP (Prerequisites)\annclassification\Annclassification_practice\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Loading Tensorboard

In [63]:
# Loading tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [66]:
%tensorboard --logdir logs/fit20241211-114716

Reusing TensorBoard on port 6006 (pid 7624), started 22:57:14 ago. (Use '!kill 7624' to kill it.)